In [1]:
import datetime as dt
import math

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import (
    _fetch_batch,
    fetch_data,
    fetch_spot_data,
)

In [2]:
from datetime import date
from bisect import bisect_left

def get_expiry(input_date, index_symbol='nifty'):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_left(expiry_dates, input_date)    
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [3]:
# bnf_pandas = pd.read_csv("../data/bnf_min.csv")
bnf_pandas = pd.read_csv("../data/nifty_min (2).csv")
# bnf_pandas = pd.read_csv("../data/fin_min.csv")
# bnf_pandas = pd.read_csv("../data/midcp_min.csv")
# bnf_pandas = pd.read_csv("../data/sensex_min.csv")
# bnf_pandas = pd.read_csv("../data/bankex_min.csv")

In [5]:
bnf_pandas.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_pandas.tail()

,index,datetime,o,h,l,c,v
777613,nifty,2025-05-30 15:25:00,24741.4000,24742.7000,24740.5000,24741.7000,0
777614,nifty,2025-05-30 15:26:00,24742.2500,24746.2000,24740.3000,24740.3000,0
777615,nifty,2025-05-30 15:27:00,24741.0500,24749.0500,24739.5000,24747.1500,0
777616,nifty,2025-05-30 15:28:00,24746.5500,24746.8000,24731.1000,24745.2500,0
777617,nifty,2025-05-30 15:29:00,24743.6000,24749.3000,24731.8500,24736.6500,0


In [6]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2019]
# bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [7]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [8]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))

In [9]:
# bnf = bnf.rename({"open": "o", "high": "h", "low": "l", "close": "c", "volume": "v"})

In [10]:
from expiries import dict_expiries

In [11]:
dict_expiries['nifty']

[datetime.datetime(2017, 1, 25, 0, 0),
 datetime.datetime(2017, 2, 23, 0, 0),
 datetime.datetime(2017, 3, 30, 0, 0),
 datetime.datetime(2017, 4, 27, 0, 0),
 datetime.datetime(2017, 5, 25, 0, 0),
 datetime.datetime(2017, 6, 29, 0, 0),
 datetime.datetime(2017, 7, 27, 0, 0),
 datetime.datetime(2017, 8, 31, 0, 0),
 datetime.datetime(2017, 9, 28, 0, 0),
 datetime.datetime(2017, 10, 26, 0, 0),
 datetime.datetime(2017, 11, 30, 0, 0),
 datetime.datetime(2017, 12, 28, 0, 0),
 datetime.datetime(2018, 1, 25, 0, 0),
 datetime.datetime(2018, 2, 22, 0, 0),
 datetime.datetime(2018, 3, 28, 0, 0),
 datetime.datetime(2018, 4, 26, 0, 0),
 datetime.datetime(2018, 5, 31, 0, 0),
 datetime.datetime(2018, 6, 28, 0, 0),
 datetime.datetime(2018, 7, 26, 0, 0),
 datetime.datetime(2018, 8, 30, 0, 0),
 datetime.datetime(2018, 9, 27, 0, 0),
 datetime.datetime(2018, 10, 25, 0, 0),
 datetime.datetime(2018, 11, 29, 0, 0),
 datetime.datetime(2018, 12, 27, 0, 0),
 datetime.datetime(2019, 1, 31, 0, 0),
 datetime.datetime(

In [12]:
def resample(data, timeframe, offset=None):
    agg_list = [
        pl.col("o").first().alias("o"),
        pl.col("h").max().alias("h"),
        pl.col("l").min().alias("l"),
        pl.col("c").last().alias("c"),
    ]
    if timeframe == '10m':
        offset = '5m'
    if timeframe == '20m':
        offset = '15m'
    
    if "v" in data.columns:
        agg_list.append(pl.col("v").sum().alias("v"))
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(agg_list)
    )


In [13]:
def generate_signals(df, ema_period=20, rsi_threshold=50):
    """
    EMA Rejection:
    Detects weakness when price closes below the EMA after testing above it intraday.
    """
    df['daily_high_till_now'] = df.groupby(df['datetime'].dt.date)['h'].cummax()
    df['EMA'] = df['c'].ewm(span=ema_period, adjust=False).mean()

    df['Sell Signal'] = (
        (df['h'] > df['EMA']) & 
        (df['c'] < df['EMA']) &
        (df['rsi'] <= rsi_threshold)
    ).astype(int)
    return df


In [14]:
def generate_trailing_signals(df, ma_period=20, n_consecutive=3):
    
    df = df.copy()
    df['ma'] = df['c'].rolling(window=ma_period).mean()

    # Create a boolean Series where MA > MA.shift(1)
    df['ma_up'] = df['ma'] > df['ma'].shift(1)

    # Use rolling window with sum to count consecutive Trues
    df['ma_up_count'] = df['ma_up'].rolling(window=n_consecutive).sum()

    # Signal = True if we have 'n' consecutive True values
    df['Trailing Signal'] = (df['ma_up_count'] == n_consecutive).astype(int)
    return df

In [15]:
import pandas as pd

def calculate_rsi(df: pd.DataFrame, period: int = 50, col: str = 'c') -> pd.Series:
    
    delta = df[col].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    df['rsi'] = rsi
    
    return df 


In [16]:
data = bnf_pandas.copy()
# data['datetime'] = pd.to_datetime(data['datetime'].dt.date)
print(data["datetime"].tail())
trading_days_set = set(data["datetime"].dt.date)
# sorted(trading_days_set)

777613   2025-05-30 15:25:00
777614   2025-05-30 15:26:00
777615   2025-05-30 15:27:00
777616   2025-05-30 15:28:00
777617   2025-05-30 15:29:00
Name: datetime, dtype: datetime64[ns]


In [17]:
# GLOBAL VARIABLES

INSTRUMENT = "NIFTY"
INDEX = "nifty"
INDEX_MROUND = 50
# INDEX_MROUND=100

# INSTRUMENT = "NIFTY"
# INDEX = "nifty"
# INDEX_MROUND=50

PORTFOLIO_VALUE = 10_00_000
INDEX_LEV = 8
RPT_CE = 0.02
RPT_PE = 0.02
SLIPPAGE = 0.01
MAX_MARGIN = 250
# TF = "10m"

# SIGNAL_MA = 20
# NUM_OF_CANDELS = 1
# T_MA=20
# TARGET=100

In [18]:
bnf = resample(bnf, '1m')
data = bnf.to_pandas()
data.tail()
# data[['MA','signal_spot']]=MA(data,200)
# data[data['signal_spot']==1].head(50)
# bnf
# data.tail(50)
# data[data['datetime'].dt.date == dt.date(2024, 4, 29)]

,datetime,o,h,l,c,v
593152,2025-05-30 15:25:00,24741.4000,24742.7000,24740.5000,24741.7000,0
593153,2025-05-30 15:26:00,24742.2500,24746.2000,24740.3000,24740.3000,0
593154,2025-05-30 15:27:00,24741.0500,24749.0500,24739.5000,24747.1500,0
593155,2025-05-30 15:28:00,24746.5500,24746.8000,24731.1000,24745.2500,0
593156,2025-05-30 15:29:00,24743.6000,24749.3000,24731.8500,24736.6500,0


In [33]:
async def ce_trade(data, tf, offset, ema, rsi_period, rsi_threshold, start_date, end_date):
    df = data.copy()
    
    # start_date = dt.date(2019, 1, 1)
    # end_date = dt.date(2025, 5, 31)
    current_date = start_date

    combined_trades = pd.DataFrame()
    total_trades = pd.DataFrame()
    time_of_day = dt.time(9, 15)
    trade_book = []
    ce_lowest_low = float("inf")
    ce_highest_high = float("-inf")
    entry_rsi = 0
    is_trailing_active = False

    while current_date < end_date:
        print(f'CE : {current_date}')
        entry = 0
        initial_sl = 0
        exit = 0
        in_ce_trade = False
        in_pe_trade = False
        # signal_exist=False

        points_captured = 0
        remark = ""
        trailing_active = False
        tsl = 0
        stop_trading = False
        is_gap_ce_sl = False
        previous_ce_sl_hit = False
        current_date_increament_flag = False
        # tsl_high = 0

        starting_time = dt.time(9, 15)

        ending_time = dt.time(15, 30)

        if not in_ce_trade and current_date in trading_days_set:

            ce_search_datetime = dt.datetime.combine(current_date, time_of_day)
            # print(f'current date : {ce_search_datetime}')

            spot_open = df.loc[df["datetime"] >= ce_search_datetime, "o"].iloc[0]
            
            spot_atm = int(
                math.ceil(spot_open / INDEX_MROUND) * INDEX_MROUND
            )  ##ROUNDS TO NEAREST 500 OTM
            
            nearest_expiry = get_expiry(current_date)
            
            selected_strike_ce = spot_atm
            # print(f'selected strike CE : {selected_strike_ce}')
            ce_df = await fetch_data(
                index=INDEX,
                start_date=nearest_expiry - dt.timedelta(days=14),
                start_time=starting_time,
                end_date=nearest_expiry,
                end_time=ending_time,
                strike=selected_strike_ce,
                asset_class="C",
                expiry=nearest_expiry,
            )
            # print(ce_df)
            if ce_df is not None and not isinstance(ce_df, str):
                # print('new data fetched CE')
                data_ce = True
                ce_df = ce_df.select(["datetime", "o", "h", "l", "c", "v"])
                ce_df = resample(ce_df, tf, offset)
                ce_df_pandas = ce_df.to_pandas()
                ce_df_pandas = ce_df_pandas[ce_df_pandas['datetime'].dt.time != dt.time(15, 30)]
                ce_df_pandas = calculate_rsi(ce_df_pandas, rsi_period)
                ce_df = generate_signals(ce_df_pandas, ema, rsi_threshold)
                # ce_df = calculate_signals(ce_df_pandas)
                # print(ce_df.to_string())
            else:
                data_ce = False
                current_date += dt.timedelta(days=1)
                continue

            if data_ce:

                for i in range(0, len(ce_df)):
                    current_candle = ce_df.iloc[i]
                    current_candle_open = ce_df.iloc[i]["o"]
                    current_candle_high = ce_df.iloc[i]["h"]
                    current_candle_low = ce_df.iloc[i]["l"]
                    current_candle_close = ce_df.iloc[i]["c"]

                    previous_candle_low = ce_df.iloc[i - 1]["l"]
                    previous_candle_close = ce_df.iloc[i - 1]["c"]
                    

                    expiry = nearest_expiry
                    strike = selected_strike_ce
                    asset_class = "C"
                    # print(ce_df.iloc[i])

                    signal = ce_df.iloc[i - 1]["Sell Signal"]
                    # candle_condition = (ce_df.iloc[i-1]['daily_high_till_now'] - previous_candle_close) > 10

                    if ce_df.iloc[i]["datetime"] >= ce_search_datetime:


                        if (
                            not previous_ce_sl_hit
                            and not in_ce_trade
                            and signal
                            # and current_candle_low < previous_candle_low
                            and ce_df.iloc[i]["datetime"].time() > time_of_day
                            and (
                                (nearest_expiry - ce_df.iloc[i]["datetime"].date()).days
                                >= 0
                                and (
                                    nearest_expiry - ce_df.iloc[i]["datetime"].date()
                                ).days
                                < 7
                            )
                            and ce_df.iloc[i]["datetime"].time() < dt.time(15, 25)
                            # and candle_condition
                        ):
                            
                            entry = current_candle_open
                            entry_date = ce_df.iloc[i-1]["datetime"].date()
                            entry_time = ce_df.iloc[i-1]["datetime"].time()
                            # initial_sl = ce_df.iloc[i - SL_CANDLES_NUM : i]["h"].max()
                            # initial_sl = day_high
                            initial_sl = ce_df.iloc[i-1]['daily_high_till_now']
                            in_ce_trade = True
                            ce_lowest_low = float("inf")
                            ce_highest_high = float("-inf")
                            # print(f'initial SL : {initial_sl}')
                            entry_rsi = ce_df.iloc[i-1]['rsi']

                            qty = RPT_CE * PORTFOLIO_VALUE / (initial_sl - entry)
                            if (
                                (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                            ) * 100 > MAX_MARGIN:
                                qty = PORTFOLIO_VALUE * INDEX_LEV / strike * (MAX_MARGIN / 100)

                            # print(f'qty : {qty}')

                        # While in trade, track the highest high and lowest low
                        if in_ce_trade:
                            # Track the highest high
                            ce_highest_high = max(ce_highest_high, current_candle_high)

                            # Track the lowest low
                            ce_lowest_low = min(ce_lowest_low, current_candle_low)

                            # if ce_lowest_low < entry * (100 - decay) / 100:
                            #     is_trailing_active = True

                            # trailing_sl_signal = ce_df['Trailing Signal'].iloc[i]

                        if (
                            in_ce_trade
                            and ce_df.iloc[i]["datetime"].time() == dt.time(9, 15)
                            and current_candle_open > initial_sl
                        ):

                            exit = current_candle_close
                            in_ce_trade = False
                            stop_trading = False
                            previous_ce_sl_hit = True
                            is_gap_ce_sl = False
                            points_captured = entry - exit
                            exit_time = ce_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            remark = "Gap SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": ce_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'RSI on Entry': entry_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": ce_df.iloc[i]["datetime"].year,
                                "Trade Month": ce_df.iloc[i]["datetime"].month,
                                "Highest High": ce_highest_high,  # Add highest high to trade data
                                "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print('apending initial sl trade')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = ce_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = ce_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            is_trailing_active = False
                            break

                        if in_ce_trade and current_candle_high > initial_sl:
                            exit = initial_sl
                            otm_datetime = ce_df.iloc[i]["datetime"]
                            in_ce_trade = False
                            stop_trading = False
                            previous_ce_sl_hit = True
                            is_gap_ce_sl = False
                            points_captured = entry - exit
                            exit_time = ce_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": ce_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'RSI on Entry': entry_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": ce_df.iloc[i]["datetime"].year,
                                "Trade Month": ce_df.iloc[i]["datetime"].month,
                                "Highest High": ce_highest_high,  # Add highest high to trade data
                                "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print('apending initial sl trade')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = ce_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = ce_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            is_trailing_active = False
                            break

                        # if in_ce_trade and is_trailing_active and trailing_sl_signal:
                        #     # print(ce_df.iloc[i])
                        #     # print(f'initial sl hit {initial_sl}')
                        #     # print(f'initial sl datetime {ce_df.iloc[i]["datetime"]}')
                        #     exit = current_candle_close
                        #     otm_datetime = ce_df.iloc[i]["datetime"]
                        #     in_ce_trade = False
                        #     stop_trading = False
                        #     previous_ce_sl_hit = True
                        #     is_gap_ce_sl = False
                        #     points_captured = entry - exit
                        #     exit_time = ce_df.iloc[i]["datetime"].time()
                        #     slippage = SLIPPAGE * (entry + exit)
                        #     pnl = qty * (points_captured - slippage)
                        #     # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                        #     remark = "TSL hit"
                        #     weekday_int = entry_date.weekday()
                        #     weekday_name = [
                        #         "Monday",
                        #         "Tuesday",
                        #         "Wednesday",
                        #         "Thursday",
                        #         "Friday",
                        #         "Saturday",
                        #         "Sunday",
                        #     ][weekday_int]
                        #     trade = {
                        #         "date": entry_date,
                        #         "day": weekday_name,
                        #         "expiry": expiry,
                        #         "DTE": (nearest_expiry - entry_date).days,
                        #         # 'atm' : atm,
                        #         # 'scrip' : index ,
                        #         "strike": strike,
                        #         "type": asset_class,
                        #         "Entry Price": entry,
                        #         "Entry Time": entry_time,
                        #         "initial sl": initial_sl,
                        #         # "TSL": tsl_high,
                        #         # 'OTM Entry' : otm_entry,
                        #         "Exit Price": exit,
                        #         "Exit date": ce_df.iloc[i]["datetime"].date(),
                        #         "Exit Time": exit_time,
                        #         'RSI on Entry': entry_rsi,
                        #         # 'OTM EXIT ' : otm_exit,
                        #         "Remark": remark,
                        #         "Points Captured": points_captured,
                        #         "Slippage": slippage,
                        #         # 'OTM cost' : otm_exit-otm_entry,
                        #         "Qty": qty,
                        #         "PnL": pnl,
                        #         "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                        #         "Trade Year": ce_df.iloc[i]["datetime"].year,
                        #         "Trade Month": ce_df.iloc[i]["datetime"].month,
                        #         "Highest High": ce_highest_high,  # Add highest high to trade data
                        #         "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                        #         "Max ROI%": (
                        #             (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                        #         )
                        #         * 100,
                        #         "Margin": (
                        #             (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                        #         )
                        #         * 100,
                        #     }
                        #     # print('apending initial sl trade')
                        #     trade_book.append(trade)
                        #     # tsl_high = 0
                        #     points_captured = 0
                        #     current_date = ce_df.iloc[i]["datetime"].date()
                        #     current_date_increament_flag = True
                        #     time_of_day = ce_df.iloc[i]["datetime"].time()
                        #     # print(f'current date changed to : {current_date} and time to {time_of_day}')
                        #     is_trailing_active = False
                        #     break

                        if (
                            in_ce_trade
                            and ce_df.iloc[i]["datetime"].date() == nearest_expiry
                            and ce_df.iloc[i]['datetime'].time() >= dt.time(15, 20)
                        ):
                            # print(ce_df.iloc[i])
                            # print(f'EOD exit {current_candle_close}')
                            # print(f'EOD datetime {ce_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            otm_datetime = ce_df.iloc[i]["datetime"]
                            in_ce_trade = False
                            previous_ce_sl_hit = True
                            is_gap_ce_sl = False
                            points_captured = entry - exit
                            exit_time = ce_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "EOD exit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": ce_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'RSI on Entry': entry_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": ce_df.iloc[i]["datetime"].year,
                                "Trade Month": ce_df.iloc[i]["datetime"].month,
                                "Highest High": ce_highest_high,  # Add highest high to trade data
                                "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print('apending EOD trade')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = nearest_expiry + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            # print(f'current date increased by 1 on expiry : {current_date}')
                            is_trailing_active = False
                            break

                        if (
                            not in_ce_trade
                            and not previous_ce_sl_hit
                            and ce_df.iloc[i]["datetime"].time() > dt.time(15, 15)
                        ):
                            # print('inside exoiry non trade date increment')
                            current_date = current_date + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            break

        if not current_date_increament_flag:
            current_date = current_date + dt.timedelta(days=1)
            current_date_increament_flag = False

    trade_book_df = pd.DataFrame(trade_book)

    return trade_book_df

In [34]:
async def pe_trade(data, tf, offset, ema, rsi_period, rsi_threshold, start_date, end_date):
    df = data.copy()
    
    # start_date = dt.date(2019, 1, 1)
    # end_date = dt.date(2025, 5, 31)
    current_date = start_date

    combined_trades = pd.DataFrame()
    total_trades = pd.DataFrame()
    time_of_day = dt.time(9, 15)
    trade_book = []
    pe_lowest_low = float("inf")
    pe_highest_high = float("-inf")
    entry_rsi = 0
    is_trailing_active = False

    while current_date < end_date:
        print(f'PE : {current_date}')
        entry = 0
        initial_sl = 0
        exit = 0
        in_pe_trade = False
        in_pe_trade = False
        # signal_exist=False

        points_captured = 0
        remark = ""
        trailing_active = False
        tsl = 0
        stop_trading = False
        is_gap_pe_sl = False
        previous_pe_sl_hit = False
        current_date_increament_flag = False
        # tsl_high = 0

        starting_time = dt.time(9, 15)

        ending_time = dt.time(15, 30)

        if not in_pe_trade and current_date in trading_days_set:

            pe_search_datetime = dt.datetime.combine(current_date, time_of_day)
            # print(f'current date : {pe_search_datetime}')

            spot_open = df.loc[df["datetime"] >= pe_search_datetime, "o"].iloc[0]
            # print(f'spot open : {spot_open}')
            # spot_atm = int(round(spot_open / INDEX_MROUND) * INDEX_MROUND)
            spot_atm = int(
                math.floor(spot_open / INDEX_MROUND) * INDEX_MROUND
            )  ##ROUNDS TO NEAREST 500 OTM
            # print(f'spot atm : {spot_atm}')
            # nearest_expiry = await get_expiry(current_date)
            nearest_expiry = get_expiry(current_date)
            # if current_date== nearest_expiry:
            #     next_expiry_passing_value = current_date + dt.timedelta(days=1)
            #     nearest_expiry = await get_expiry_nifty( next_expiry_passing_value)
            # print(f'passing date for expry : {current_date}')
            # nearest_expiry = await get_monthly_expiry_nifty(current_date)
            # print(f'nearest expiry{nearest_expiry}')
            selected_strike_pe = spot_atm
            # print(f'selected strike PE : {selected_strike_pe}')
            pe_df = await fetch_data(
                index=INDEX,
                start_date=nearest_expiry - dt.timedelta(days=14),
                start_time=starting_time,
                end_date=nearest_expiry,
                end_time=ending_time,
                strike=selected_strike_pe,
                asset_class="P",
                expiry=nearest_expiry,
            )
            if pe_df is not None and not isinstance(pe_df, str):
                # print('new data fetched PE')
                data_pe = True
                pe_df = pe_df.select(["datetime", "o", "h", "l", "c", "v"])
                pe_df = resample(pe_df, tf, offset)
                pe_df_pandas = pe_df.to_pandas()
                pe_df_pandas = pe_df_pandas[pe_df_pandas['datetime'].dt.time != dt.time(15, 30)]
                pe_df_pandas = calculate_rsi(pe_df_pandas, rsi_period)
                pe_df = generate_signals(pe_df_pandas, ema, rsi_threshold)
                # pe_df = calculate_signals(pe_df_pandas)
                # print(pe_df.to_string())
            else:
                data_pe = False
                current_date += dt.timedelta(days=1)
                continue

            if data_pe:

                for i in range(0, len(pe_df)):
                    current_candle = pe_df.iloc[i]
                    current_candle_open = pe_df.iloc[i]["o"]
                    current_candle_high = pe_df.iloc[i]["h"]
                    current_candle_low = pe_df.iloc[i]["l"]
                    current_candle_close = pe_df.iloc[i]["c"]

                    previous_candle_low = pe_df.iloc[i - 1]["l"]
                    previous_candle_close = pe_df.iloc[i - 1]["c"]
                    

                    expiry = nearest_expiry
                    strike = selected_strike_pe
                    asset_class = "P"
                    # print(pe_df.iloc[i])

                    signal = pe_df.iloc[i - 1]["Sell Signal"]
                    # candle_condition = (pe_df.iloc[i-1]['daily_high_till_now'] - previous_candle_close) > 10

                    if pe_df.iloc[i]["datetime"] >= pe_search_datetime:


                        if (
                            not previous_pe_sl_hit
                            and not in_pe_trade
                            and signal
                            # and current_candle_low < previous_candle_low
                            and pe_df.iloc[i]["datetime"].time() > time_of_day
                            and (
                                (nearest_expiry - pe_df.iloc[i]["datetime"].date()).days
                                >= 0
                                and (
                                    nearest_expiry - pe_df.iloc[i]["datetime"].date()
                                ).days
                                < 7
                            )
                            and pe_df.iloc[i]["datetime"].time() < dt.time(15, 25)
                            # and candle_condition
                        ):
                            # print(pe_df.iloc[i-1])
                            # print(f'entry found {previous_candle_low}')
                            # print(f'entry datetime {pe_df.iloc[i]["datetime"]}')
                            
                            # today_data = pe_df[pe_df['datetime'].dt.date == current_candle['datetime'].date()]
                            # day_high = today_data.iloc[0 : i]['h'].max()
                            # print(today_data.to_string())
                            
                            entry = current_candle_open
                            entry_date = pe_df.iloc[i-1]["datetime"].date()
                            entry_time = pe_df.iloc[i-1]["datetime"].time()
                            # initial_sl = pe_df.iloc[i - SL_CANDLES_NUM : i]["h"].max()
                            # initial_sl = day_high
                            initial_sl = pe_df.iloc[i-1]['daily_high_till_now']
                            in_pe_trade = True
                            pe_lowest_low = float("inf")
                            pe_highest_high = float("-inf")
                            entry_rsi = pe_df.iloc[i-1]['rsi']

                            qty = RPT_PE * PORTFOLIO_VALUE / (initial_sl - entry)
                            if (
                                (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                            ) * 100 > MAX_MARGIN:
                                qty = PORTFOLIO_VALUE * INDEX_LEV / strike * (MAX_MARGIN / 100)

                        # While in trade, track the highest high and lowest low
                        if in_pe_trade:
                            # Track the highest high
                            pe_highest_high = max(pe_highest_high, current_candle_high)

                            # Track the lowest low
                            pe_lowest_low = min(pe_lowest_low, current_candle_low)
                            
                            # if pe_lowest_low < entry * (100 - decay) / 100:
                            #     is_trailing_active = True

                            # trailing_sl_signal = pe_df['Trailing Signal'].iloc[i]
                           
                        if (
                            in_pe_trade
                            and pe_df.iloc[i]["datetime"].time() == dt.time(9, 15)
                            and current_candle_open > initial_sl
                        ):

                            # print(pe_df.iloc[i])
                            # print(f'GAP sl hit {initial_sl}')
                            # print(f'GAP sl datetime {pe_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            in_pe_trade = False
                            stop_trading = False
                            previous_pe_sl_hit = True
                            is_gap_pe_sl = False
                            points_captured = entry - exit
                            exit_time = pe_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            remark = "Gap SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": pe_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'RSI on Entry': entry_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": pe_df.iloc[i]["datetime"].year,
                                "Trade Month": pe_df.iloc[i]["datetime"].month,
                                "Highest High": pe_highest_high,  # Add highest high to trade data
                                "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print('apending initial sl trade')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = pe_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = pe_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            is_trailing_active = False
                            break

                        if in_pe_trade and current_candle_high > initial_sl:
                            # print(pe_df.iloc[i])
                            # print(f'initial sl hit {initial_sl}')
                            # print(f'initial sl datetime {pe_df.iloc[i]["datetime"]}')
                            exit = initial_sl
                            otm_datetime = pe_df.iloc[i]["datetime"]
                            in_pe_trade = False
                            stop_trading = False
                            previous_pe_sl_hit = True
                            is_gap_pe_sl = False
                            points_captured = entry - exit
                            exit_time = pe_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": pe_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'RSI on Entry': entry_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": pe_df.iloc[i]["datetime"].year,
                                "Trade Month": pe_df.iloc[i]["datetime"].month,
                                "Highest High": pe_highest_high,  # Add highest high to trade data
                                "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print('apending initial sl trade')
                            trade_book.append(trade)
                            tsl_high = 0
                            points_captured = 0
                            current_date = pe_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = pe_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            is_trailing_active = False
                            break

                        # if in_pe_trade and is_trailing_active and trailing_sl_signal:
                        #     # print(pe_df.iloc[i])
                        #     # print(f'initial sl hit {initial_sl}')
                        #     # print(f'initial sl datetime {pe_df.iloc[i]["datetime"]}')
                        #     exit = current_candle_close
                        #     otm_datetime = pe_df.iloc[i]["datetime"]
                        #     in_pe_trade = False
                        #     stop_trading = False
                        #     previous_pe_sl_hit = True
                        #     is_gap_pe_sl = False
                        #     points_captured = entry - exit
                        #     exit_time = pe_df.iloc[i]["datetime"].time()
                        #     slippage = SLIPPAGE * (entry + exit)
                        #     pnl = qty * (points_captured - slippage)
                        #     # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                        #     remark = "TSL hit"
                        #     weekday_int = entry_date.weekday()
                        #     weekday_name = [
                        #         "Monday",
                        #         "Tuesday",
                        #         "Wednesday",
                        #         "Thursday",
                        #         "Friday",
                        #         "Saturday",
                        #         "Sunday",
                        #     ][weekday_int]
                        #     trade = {
                        #         "date": entry_date,
                        #         "day": weekday_name,
                        #         "expiry": expiry,
                        #         "DTE": (nearest_expiry - entry_date).days,
                        #         # 'atm' : atm,
                        #         # 'scrip' : index ,
                        #         "strike": strike,
                        #         "type": asset_class,
                        #         "Entry Price": entry,
                        #         "Entry Time": entry_time,
                        #         "initial sl": initial_sl,
                        #         # "TSL": tsl_high,
                        #         # 'OTM Entry' : otm_entry,
                        #         "Exit Price": exit,
                        #         "Exit date": pe_df.iloc[i]["datetime"].date(),
                        #         "Exit Time": exit_time,
                        #         'RSI on Entry': entry_rsi,
                        #         # 'OTM EXIT ' : otm_exit,
                        #         "Remark": remark,
                        #         "Points Captured": points_captured,
                        #         "Slippage": slippage,
                        #         # 'OTM cost' : otm_exit-otm_entry,
                        #         "Qty": qty,
                        #         "PnL": pnl,
                        #         "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                        #         "Trade Year": pe_df.iloc[i]["datetime"].year,
                        #         "Trade Month": pe_df.iloc[i]["datetime"].month,
                        #         "Highest High": pe_highest_high,  # Add highest high to trade data
                        #         "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                        #         "Max ROI%": (
                        #             (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                        #         )
                        #         * 100,
                        #         "Margin": (
                        #             (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                        #         )
                        #         * 100,
                        #     }
                        #     # print('apending initial sl trade')
                        #     trade_book.append(trade)
                        #     tsl_high = 0
                        #     points_captured = 0
                        #     current_date = pe_df.iloc[i]["datetime"].date()
                        #     current_date_increament_flag = True
                        #     time_of_day = pe_df.iloc[i]["datetime"].time()
                        #     # print(f'current date changed to : {current_date} and time to {time_of_day}')
                        #     is_trailing_active = False
                        #     break
                            
                        if (
                            in_pe_trade
                            and pe_df.iloc[i]["datetime"].date() == nearest_expiry
                            and pe_df.iloc[i]['datetime'].time() >= dt.time(15, 20)
                        ):
                            # print(pe_df.iloc[i])
                            # print(f'EOD exit {current_candle_close}')
                            # print(f'EOD datetime {pe_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            otm_datetime = pe_df.iloc[i]["datetime"]
                            in_pe_trade = False
                            previous_pe_sl_hit = True
                            is_gap_pe_sl = False
                            points_captured = entry - exit
                            exit_time = pe_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "EOD exit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": pe_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'RSI on Entry': entry_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": pe_df.iloc[i]["datetime"].year,
                                "Trade Month": pe_df.iloc[i]["datetime"].month,
                                "Highest High": pe_highest_high,  # Add highest high to trade data
                                "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print('apending EOD trade')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = nearest_expiry + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            # print(f'current date increased by 1 on expiry : {current_date}')
                            is_trailing_active = False
                            break

                        if (
                            not in_pe_trade
                            and not previous_pe_sl_hit
                            and pe_df.iloc[i]["datetime"].time() > dt.time(15, 15)
                        ):
                            # print('inside exoiry non trade date increment')
                            current_date = current_date + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            break

        if not current_date_increament_flag:
            current_date = current_date + dt.timedelta(days=1)
            current_date_increament_flag = False

    trade_book_df = pd.DataFrame(trade_book)

    return trade_book_df

In [35]:
# tb_ce = pd.DataFrame()
# tb_pe = pd.DataFrame()

async def execute(DF, tf, offset, EMA, rsi_period, rsi_threshold, start_date, end_date):
    data = DF.copy()
    tb_ce = await ce_trade(data, tf, offset, EMA, rsi_period, rsi_threshold, start_date, end_date)
    tb_pe = await pe_trade(data, tf, offset, EMA, rsi_period, rsi_threshold, start_date, end_date)
    tb = pd.concat([tb_ce, tb_pe], ignore_index=True)
    # print(len(tb))
    if len(tb)>0:
        tb = tb.sort_values(by="date")
    return tb

In [36]:
def generate_stats(tb_expiry, ema_window):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{ema_window}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [37]:
tf1 = '10m'
offset1 = '5m'
ema1 = 18
# decay_ = 80
# trailing_ma_ = 18
rsi_period = 15
rsi_threshold = 50
bnf = resample(bnf, tf1)
data = bnf.to_pandas()

start_date = dt.date(2019, 1, 1)
end_date = dt.date(2025, 5, 31)

tb = await execute(data, tf1, offset1, ema1, rsi_period, rsi_threshold, start_date, end_date)

CE : 2019-01-01
CE : 2019-01-02
CE : 2019-01-03
CE : 2019-01-04
CE : 2019-01-05
CE : 2019-01-06
CE : 2019-01-07
CE : 2019-01-08
CE : 2019-01-09
CE : 2019-01-10
CE : 2019-01-11
CE : 2019-01-12
CE : 2019-01-13
CE : 2019-01-14
CE : 2019-01-15
CE : 2019-01-16
CE : 2019-01-17
CE : 2019-01-18
CE : 2019-01-19
CE : 2019-01-20
CE : 2019-01-21
CE : 2019-01-22
CE : 2019-01-23
CE : 2019-01-24
CE : 2019-01-25
CE : 2019-02-01
CE : 2019-02-15
CE : 2019-02-22
CE : 2019-02-22
CE : 2019-02-25
CE : 2019-02-25
CE : 2019-02-26
CE : 2019-02-26
CE : 2019-02-27
CE : 2019-03-01
CE : 2019-03-05
CE : 2019-03-06
CE : 2019-03-07
CE : 2019-03-08
CE : 2019-03-11
CE : 2019-03-11
CE : 2019-03-12
CE : 2019-03-13
CE : 2019-03-13
CE : 2019-03-14
CE : 2019-03-15
CE : 2019-03-16
CE : 2019-03-17
CE : 2019-03-18
CE : 2019-03-21
CE : 2019-03-22
CE : 2019-03-29
CE : 2019-04-01
CE : 2019-04-05
CE : 2019-04-05
CE : 2019-04-06
CE : 2019-04-07
CE : 2019-04-08
CE : 2019-04-12
CE : 2019-04-12
CE : 2019-04-13
CE : 2019-04-14
CE : 201

In [38]:
tb['DATETIME'] = pd.to_datetime(tb['date'].astype(str) + ' ' + tb['Entry Time'].astype(str))

In [39]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, 'Hammer Trade-wise')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,175.7652,207,42.5121,4.6963,-1.9959,-14.0627,12.4987,Hammer Trade-wise
2020,120.2752,244,40.1639,4.5473,-2.2285,-20.5674,5.8479,Hammer Trade-wise
2021,121.7279,221,43.4389,4.1995,-2.2514,-14.1298,8.6150,Hammer Trade-wise
2022,111.1433,245,37.1429,5.2523,-2.3819,-21.0279,5.2855,Hammer Trade-wise
2023,78.3279,218,38.9908,4.1295,-2.0502,-16.1907,4.8378,Hammer Trade-wise
2024,82.6389,232,37.0690,4.6055,-2.1468,-23.5091,3.5152,Hammer Trade-wise
2025,29.2536,89,43.8202,4.0198,-2.5504,-23.5709,1.2411,Hammer Trade-wise
Overall,719.1321,1456,40.0412,4.5349,-2.2047,-24.8668,28.9194,Hammer Trade-wise


In [39]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, 'Hammer Trade-wise')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,175.7652,207,42.5121,4.6963,-1.9959,-14.0627,12.4987,Hammer Trade-wise
2020,120.2752,244,40.1639,4.5473,-2.2285,-20.5674,5.8479,Hammer Trade-wise
2021,121.7279,221,43.4389,4.1995,-2.2514,-14.1298,8.6150,Hammer Trade-wise
2022,111.1433,245,37.1429,5.2523,-2.3819,-21.0279,5.2855,Hammer Trade-wise
2023,78.3279,218,38.9908,4.1295,-2.0502,-16.1907,4.8378,Hammer Trade-wise
2024,82.6389,232,37.0690,4.6055,-2.1468,-23.5091,3.5152,Hammer Trade-wise
2025,35.8111,63,41.2698,4.5832,-2.2528,-10.7521,3.3306,Hammer Trade-wise
Overall,725.6896,1430,39.8601,4.5724,-2.1867,-24.8668,29.1831,Hammer Trade-wise


In [41]:
# tb.to_csv('Hammer_10m_15_50.csv', index=False)

In [24]:
weekly_roi = tb.groupby('expiry')['ROI%'].sum().reset_index()
weekly_roi['expiry'] = pd.to_datetime(weekly_roi['expiry'])
weekly_roi['Trade Year'] = weekly_roi['expiry'].dt.year
weekly_roi

,expiry,ROI%,Trade Year
0,2025-05-08,6.1108,2025
1,2025-05-15,-6.9257,2025
2,2025-05-22,-0.4983,2025
3,2025-05-29,5.8240,2025


In [98]:
stats = generate_stats(weekly_roi, 'Hammer Weekly')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,165.8776,47,70.2128,6.4200,-3.2845,-8.9285,18.5785,Hammer Weekly
2020,113.7935,53,62.2642,5.4292,-3.2685,-14.4217,7.8904,Hammer Weekly
2021,111.9018,52,71.1538,4.5305,-3.7151,-8.7373,12.8074,Hammer Weekly
2022,108.0109,52,53.8462,7.4428,-4.1828,-16.9439,6.3746,Hammer Weekly
2023,78.0217,51,58.8235,4.4648,-2.6630,-13.4325,5.8084,Hammer Weekly
2024,84.3846,52,57.6923,5.0805,-3.0923,-17.8927,4.7161,Hammer Weekly
2025,27.0750,14,57.1429,5.4726,-2.7843,-7.5346,3.5934,Hammer Weekly
Overall,689.0651,321,61.9938,5.5135,-3.3453,-21.4835,32.0742,Hammer Weekly


In [29]:
tb.tail(25)

,date,day,expiry,DTE,strike,type,Entry Price,Entry Time,initial sl,Exit Price,Exit date,Exit Time,RSI on Entry,Remark,Points Captured,Slippage,Qty,PnL,ROI%,Trade Year,Trade Month,Highest High,Lowest Low,Max ROI%,Margin,DATETIME
7,2025-05-02,Friday,2025-05-08,6,24300,P,138.9500,09:15:00,234.0000,26.4000,2025-05-08,15:25:00,30.6903,EOD exit,112.5500,1.6535,210.4156,23334.3503,2.3334,2025,5,232.4000,1.5500,2.8911,63.9137,2025-05-02 09:15:00
0,2025-05-02,Friday,2025-05-08,6,24350,C,187.6500,14:05:00,395.9500,0.0500,2025-05-08,15:25:00,48.7650,EOD exit,187.6000,1.8770,96.0154,17832.2612,1.7832,2025,5,258.1500,0.0500,1.8012,29.2247,2025-05-02 14:05:00
8,2025-05-09,Friday,2025-05-15,6,23900,P,183.0500,11:25:00,330.0000,0.0500,2025-05-15,15:25:00,46.3958,EOD exit,183.0000,1.8310,136.1007,24657.2304,2.4657,2025,5,240.8500,0.0500,2.4906,40.6601,2025-05-09 11:25:00
1,2025-05-09,Friday,2025-05-15,6,23950,C,308.2500,11:35:00,403.7000,647.7500,2025-05-12,09:15:00,37.7299,Gap SL hit,-339.5000,9.5600,209.5338,-73139.8638,-7.3140,2025,5,663.4500,231.1500,1.6155,62.7292,2025-05-09 11:35:00
2,2025-05-13,Tuesday,2025-05-15,2,24900,C,131.9500,09:15:00,223.8000,223.8000,2025-05-15,14:05:00,49.0729,SL hit,-91.8500,3.5575,217.7463,-20774.6326,-2.0775,2025,5,231.7500,5.0000,2.7643,67.7735,2025-05-13 09:15:00
3,2025-05-16,Friday,2025-05-22,6,25100,C,151.2000,09:35:00,191.8000,0.0500,2025-05-22,15:25:00,49.9519,EOD exit,151.1500,1.5125,492.6108,73713.0542,7.3713,2025,5,167.1500,0.0500,7.4458,154.5567,2025-05-16 09:35:00
9,2025-05-16,Friday,2025-05-22,6,25050,P,213.2500,12:15:00,238.9000,238.9000,2025-05-19,14:05:00,43.9824,SL hit,-25.6500,4.5215,779.7271,-23525.5361,-2.3526,2025,5,245.0000,158.8000,4.2456,244.1520,2025-05-16 12:15:00
10,2025-05-20,Tuesday,2025-05-22,2,24950,P,149.1000,10:05:00,189.0000,189.0000,2025-05-20,12:45:00,46.7232,SL hit,-39.9000,3.3810,501.2531,-21694.7368,-2.1695,2025,5,210.0000,114.5000,1.7343,156.3283,2025-05-20 10:05:00
11,2025-05-21,Wednesday,2025-05-22,1,24700,P,61.2000,11:45:00,112.6500,112.6500,2025-05-21,12:05:00,26.8852,SL hit,-51.4500,1.7385,388.7269,-20675.8017,-2.0676,2025,5,113.7500,57.4500,0.1458,120.0194,2025-05-21 11:45:00
12,2025-05-21,Wednesday,2025-05-22,1,24750,P,91.0500,14:45:00,165.7000,165.7000,2025-05-22,09:15:00,43.6575,SL hit,-74.6500,2.5675,267.9169,-20687.8768,-2.0688,2025,5,233.7000,82.6000,0.2264,82.8868,2025-05-21 14:45:00


In [28]:
tb_ce = tb[tb['type'] == 'C']
tb_pe = tb[tb['type'] == 'P']

In [29]:
# tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb_ce, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,61.3045,112,33.9286,5.3942,-1.9415,-26.8144,2.2863,...
2020,36.9525,128,33.5938,5.1875,-2.1895,-62.3433,0.5927,...
2021,18.5655,122,33.6066,4.8782,-2.2400,-27.6852,0.6706,...
2022,28.9205,119,30.2521,6.3562,-2.4085,-33.3810,0.8664,...
2023,18.3090,121,29.7521,5.0317,-1.9157,-41.2694,0.4436,...
2024,45.7616,110,32.7273,5.5123,-2.0633,-48.4864,0.9438,...
2025,43.8320,23,43.4783,7.2385,-2.1964,-11.6965,3.7475,...
Overall,253.6456,735,32.6531,5.4535,-2.1317,-70.7034,3.5875,...


In [30]:
# tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb_pe, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,69.6642,109,35.7798,5.4556,-2.0444,-24.7296,2.8170,...
2020,114.0508,116,38.7931,6.3034,-2.3888,-39.8218,2.8640,...
2021,64.5999,107,40.1869,4.8307,-2.2363,-19.9513,3.2379,...
2022,90.7670,127,34.6457,6.4393,-2.3200,-48.0643,1.8884,...
2023,42.1746,117,33.3333,5.0981,-2.0083,-35.0449,1.2034,...
2024,8.5159,110,32.7273,4.9054,-2.2713,-47.5886,0.1789,...
2025,-7.1063,37,35.1351,3.4161,-2.1465,-16.6179,-0.4276,...
Overall,382.6661,723,35.8230,5.4336,-2.2083,-54.7600,6.9881,...


In [27]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,130.9686,221,34.8416,5.4253,-1.9915,-23.6824,5.5302,...
2020,151.0034,244,36.0656,5.7581,-2.2802,-26.4993,5.6984,...
2021,83.1654,229,36.6812,4.8539,-2.2384,-22.4713,3.7010,...
2022,119.6875,246,32.5203,6.4019,-2.3642,-28.1178,4.2566,...
2023,60.4836,238,31.5126,5.0662,-1.9600,-20.8403,2.9022,...
2024,58.5436,218,33.0275,5.2089,-2.1678,-25.1252,2.3301,...
2025,44.2623,45,37.7778,6.1039,-2.1252,-11.1397,3.9734,...
Overall,648.1145,1441,34.2124,5.4830,-2.1677,-28.1178,23.0500,...


In [26]:
tb_margin = tb[tb['Margin'] > 100]
tb_margin['ROI%'].sum(), len(tb_margin), len(tb)

(534.3461430674945, 1053, 1458)

In [25]:
# tb.to_csv('EMAR_5m_50EMA.csv', index=False)

In [27]:
# tb.tail(50)

In [37]:
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,113.1501,230,34.3478,5.0277,-1.8810,-22.5788,5.0113,...
2020,134.9478,256,34.3750,5.8342,-2.2527,-35.2967,3.8232,...
2021,97.6609,233,36.0515,4.9230,-2.1199,-21.8114,4.4775,...
2022,103.6360,248,31.8548,6.3140,-2.3383,-36.9616,2.8039,...
2023,119.8259,217,35.9447,4.9810,-1.9330,-14.0168,8.5487,...
2024,45.9805,231,32.0346,4.9413,-2.0362,-31.7348,1.4489,...
2025,46.0862,44,38.6364,5.9487,-2.0386,-8.8266,5.2213,...
Overall,661.2874,1459,34.2015,5.3672,-2.1010,-36.9616,17.8912,...


# Simulation

In [ ]:
TF_ = ['10m']
EMA_ = [18]
RSI_ = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
# MA_Range = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
RSI_THRESHOLD_ = [30, 40, 50, 60, 70, 80]

for tf in TF_:
    for ema in EMA_:
        for rsi in RSI_:
            for rsi_threshold in RSI_THRESHOLD_:
                variation = f'5m, 50EMA, RSI: {rsi}, RSI Threshold: {rsi_threshold}'
                print(variation)
                if tf == '10m':
                    offset = '5m'
                elif tf >= '20m':
                    offset = '15m'
                else:
                    offset = '0m'
                bnfx = resample(bnf, tf, offset)
                data = bnfx.to_pandas()
                tb_ce = await ce_trade(data, tf, offset, ema, rsi, rsi_threshold)
                tb_pe = await pe_trade(data, tf, offset, ema, rsi, rsi_threshold)
                tb = pd.concat([tb_ce, tb_pe], ignore_index=True)

                if len(tb) > 0:
                    tb = tb.sort_values(by="date")
                    tb['DATETIME'] = pd.to_datetime(tb['date'].astype(str) + ' ' + tb['Entry Time'].astype(str))
                    tb = tb.sort_values(by='DATETIME')
                    stats = generate_stats(tb, variation)
                    for x, y in stats.items():
                        z = pd.DataFrame(y)
                        print(z.to_string())

                        

5m, 50EMA, RSI: 5, RSI Threshold: 30


In [40]:
tb.to_csv('Hammer.csv', index=False)